# Features

    · id: only for test & sample submission files, id for prediction sample identification
    
    · price: price in USD
    
    · carat: weight of the diamond
    
    · cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)
    
    · color: diamond colour, from J (worst) to D (best)
    
    · clarity: a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
    
    · x: length in mm
    
    · y: width in mm
    
    · z: depth in mm
    
    · depth: total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
    
    · table: width of top of diamond relative to widest point (43--95)


In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 


In [2]:
diamond=pd.read_csv('diamonds_train.csv')
diamond.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


In [3]:
X=diamond.drop(columns=['price'])
y=diamond.price

In [4]:
clarity={'I1':0, 'SI2':1, 'SI1':2, 'VS2':3, 'VS1':4, 'VVS2':5, 'VVS1':6, 'IF':7}
cut={'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}
color={'J':0, 'I':1, 'H':2, 'G':3, 'F':4, 'E':5, 'D':6}

In [5]:
def labeling(s, dic):
    return dic[s]

In [6]:
X.clarity=X.clarity.apply(lambda x: labeling(x, clarity))
X.cut=X.cut.apply(lambda x: labeling(x, cut))
X.color=X.color.apply(lambda x: labeling(x, color))

In [7]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.21,3,0,3,62.4,58.0,6.83,6.79,4.25
1,0.32,2,2,3,63.0,57.0,4.35,4.38,2.75
2,0.71,0,3,4,65.5,55.0,5.62,5.53,3.65
3,0.41,1,6,2,63.8,56.0,4.68,4.72,3.00
4,1.02,4,3,2,60.5,59.0,6.55,6.51,3.95


In [8]:
X=X.drop(columns=['table'])
X.head()

,carat,cut,color,clarity,depth,x,y,z
0,1.21,3,0,3,62.4,6.83,6.79,4.25
1,0.32,2,2,3,63.0,4.35,4.38,2.75
2,0.71,0,3,4,65.5,5.62,5.53,3.65
3,0.41,1,6,2,63.8,4.68,4.72,3.00
4,1.02,4,3,2,60.5,6.55,6.51,3.95


In [9]:
X['Vol']=X.x*X.y*X.z
X['Sum']=X.carat**3+X.color+2*X.clarity
X.head()

,carat,cut,color,clarity,depth,x,y,z,Vol,Sum
0,1.21,3,0,3,62.4,6.83,6.79,4.25,197.096725,7.771561
1,0.32,2,2,3,63.0,4.35,4.38,2.75,52.395750,8.032768
2,0.71,0,3,4,65.5,5.62,5.53,3.65,113.436890,11.357911
3,0.41,1,6,2,63.8,4.68,4.72,3.00,66.268800,10.068921
4,1.02,4,3,2,60.5,6.55,6.51,3.95,168.429975,8.061208


In [10]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
gbm = lgb.LGBMRegressor(num_leaves=100,
                        learning_rate=0.05,
                        n_estimators=200)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=50, verbose=False)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.05, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=100, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [12]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
print('RMSE:', mean_squared_error(y_test, y_pred) ** 0.5)

RMSE: 525.2347754106627
